# Storage Systems and Demand-Side Management

This tutorial covers how to model storage systems and demand-side management in PyPSA. We'll explore different types of storage, their characteristics, and how to optimize their operation.

## Types of Storage

PyPSA supports various types of storage systems:

1. **Electrical Storage** (e.g., batteries)
2. **Thermal Storage** (e.g., heat storage)
3. **Hydro Storage** (e.g., pumped hydro)
4. **Chemical Storage** (e.g., hydrogen)

Let's create a network with different storage types:

In [ ]:
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Create network with time series
network = pypsa.Network()
network.set_snapshots(pd.date_range("2024-01-01", periods=24, freq="h"))

# Add buses
network.add("Bus", "electricity", v_nom=20.0)
network.add("Bus", "heat", carrier="heat")
network.add("Bus", "hydrogen", carrier="hydrogen")

# Add carriers
network.add("Carrier", "gas", co2_emissions=0.2)
network.add("Carrier", "solar")
network.add("Carrier", "heat")
network.add("Carrier", "hydrogen")

## Adding Storage Components

Let's add different types of storage systems:

In [ ]:
# Add generators
network.add(
    "Generator",
    "solar",
    bus="electricity",
    p_nom=100,
    p_max_pu=pd.Series(
        [0.0, 0.0, 0.0, 0.0, 0.1, 0.3, 0.5, 0.7, 0.8, 0.9, 0.9, 0.8,
         0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
        index=network.snapshots
    ),
    carrier="solar"
)

# Add battery storage
network.add(
    "Store",
    "battery",
    bus="electricity",
    e_nom=100,  # Energy capacity
    e_nom_extendable=True,
    e_cyclic=True,  # Allow cyclic operation
    e_min_pu=0.1,  # Minimum energy level
    p_nom=20,  # Power capacity
    p_nom_extendable=True,
    capital_cost_e=100,  # Investment cost per MWh
    capital_cost_p=1000,  # Investment cost per MW
    marginal_cost=0.1  # Operating cost per MWh
)

# Add heat storage
network.add(
    "Store",
    "heat_storage",
    bus="heat",
    e_nom=200,
    e_nom_extendable=True,
    e_cyclic=True,
    e_min_pu=0.1,
    p_nom=40,
    p_nom_extendable=True,
    capital_cost_e=50,
    capital_cost_p=500,
    standing_loss=0.01  # Heat loss per hour
)

# Add hydrogen storage
network.add(
    "Store",
    "hydrogen_storage",
    bus="hydrogen",
    e_nom=1000,
    e_nom_extendable=True,
    e_cyclic=True,
    e_min_pu=0.1,
    p_nom=100,
    p_nom_extendable=True,
    capital_cost_e=200,
    capital_cost_p=2000,
    standing_loss=0.001  # Leakage per hour
)

## Adding Power-to-X and X-to-Power Components

Let's add components to convert between different energy carriers:

In [ ]:
# Add electrolyzer (Power-to-Hydrogen)
network.add(
    "Link",
    "electrolyzer",
    bus0="electricity",
    bus1="hydrogen",
    p_nom=50,
    p_nom_extendable=True,
    efficiency=0.7,
    capital_cost=1000
)

# Add fuel cell (Hydrogen-to-Power)
network.add(
    "Link",
    "fuel_cell",
    bus0="hydrogen",
    bus1="electricity",
    p_nom=50,
    p_nom_extendable=True,
    efficiency=0.6,
    capital_cost=1000
)

# Add heat pump (Power-to-Heat)
network.add(
    "Link",
    "heat_pump",
    bus0="electricity",
    bus1="heat",
    p_nom=30,
    p_nom_extendable=True,
    efficiency=3.0,  # Coefficient of Performance
    capital_cost=800
)

## Adding Loads and Demand-Side Management

Let's add loads with different flexibility options:

In [ ]:
# Add fixed electricity load
network.add(
    "Load",
    "electricity_load",
    bus="electricity",
    p_set=50
)

# Add flexible heat load
network.add(
    "Load",
    "heat_load",
    bus="heat",
    p_set=30,
    p_max_pu=1.2,  # Maximum 20% above nominal load
    p_min_pu=0.8   # Minimum 20% below nominal load
)

# Add flexible hydrogen load
network.add(
    "Load",
    "hydrogen_load",
    bus="hydrogen",
    p_set=20,
    p_max_pu=1.5,  # Maximum 50% above nominal load
    p_min_pu=0.5   # Minimum 50% below nominal load
)

## Optimizing Storage Operation

Let's solve the optimization problem and analyze the results:

In [ ]:
# Solve the optimization problem
network.lopf()

# Plot storage operation
plt.figure(figsize=(12, 6))
network.stores_t.e.plot()
plt.title('Storage Energy Levels Over Time')
plt.xlabel('Time')
plt.ylabel('Energy (MWh)')
plt.legend(title='Storage')
plt.grid(True)
plt.show()

# Plot power flows
plt.figure(figsize=(12, 6))
network.links_t.p0.plot()
plt.title('Power Flows in Power-to-X Components')
plt.xlabel('Time')
plt.ylabel('Power (MW)')
plt.legend(title='Component')
plt.grid(True)
plt.show()

## Key Storage and Balancing Concepts

1. **Storage Characteristics**
   - Energy capacity (e_nom)
   - Power capacity (p_nom)
   - Efficiency (charging/discharging)
   - Standing losses
   - Minimum energy levels

2. **Power-to-X Technologies**
   - Convert electricity to other carriers
   - Examples: electrolyzers, heat pumps
   - Efficiency losses in conversion

3. **Demand-Side Management**
   - Flexible loads
   - Time-shifting capability
   - Maximum/minimum power limits

4. **Storage Operation**
   - Charging/discharging patterns
   - Energy level variations
   - Integration with renewables

## Next Steps

In the next tutorial, we'll explore sector coupling in more detail, including the integration of power, heat, and transport sectors.